# Webscraping da CVM

Esse robô será feito com as seguintes etapas:

1. Função init


2. Função webscraping

    a. Verificação do diretório de destino e limpeza, caso seja necessário
    
    b. Acionar o nosso webdriver
    
    c. Determinar o intervalo de tempo que queremos baixar os arquivos
    
    d. Solicitar que o webdriver baixe os arquivos
    
    e. Vamos retirar os arquivos .csv de dentro das pastas zipadas e vamos apagar as pastas zipadas
    
    
3. Função importação

    a. Determinar quais arquivos serão abertos
    
    b. Concatenar tudo num data frame só
    
    c. Fazer adaptações necessárias e retornar o data frame final
    
    


# 1. Bibliotecas

In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import zipfile
import pandas as pd
import os

# 2. Classe

In [110]:
class CVM_Extractor:

    def __init__(self, ini, fim):
        self.dir_download = os.getcwd() + '\\files' # Diretório onde está o script
        self.url = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
        self.ini = ini
        self.fim = fim

    def scraping_df(self):
        if os.path.isdir(self.dir_download): #Verificamos se o diretório existe
            print("Diretório Existe")
            # Caso o diretório exista, vamos limpá-lo se houver arquivos
            if(len(os.listdir(self.dir_download))>0):
                for i in os.listdir(self.dir_download):
                    print("Removendo arquivo ", i)
                    os.remove(f'{self.dir_download}/{i}')

        else:
            print("Diretório não existe")
            # Cria diretório que não existia
            os.mkdir(self.dir_download)

        chrome_options= webdriver.ChromeOptions()
        prefs = {'download.default_directory':self.dir_download}
        chrome_options.add_experimental_option('prefs', prefs)
        driver = webdriver.Chrome(options=chrome_options)

        print('Passo 1 de 4 - Extração dos arquivos zipados da CVM')
        driver.get(self.url)

        ano_ini = int(self.ini[-4:])
        ano_fim = int(self.fim[-4:])
        c = 2 + (ano_ini - 2011)
        cf = 2 + (ano_fim - 2011)

        while True:
            try:
                driver.find_element(By.XPATH, f'/html/body/div[1]/pre/a[{c}]').click()
                time.sleep(3)
                c += 1
                if c == cf:
                    break
            except:
                break

        while True:
            if len([x for x in os.listdir(self.dir_download) if ('.zip.crdownload' in x)]) > 0:
                pass
            else:
                break

        print('Passo 2 de 4 - Download dos arquivos concluído')
        print('Passo 3 de 4 - Iniciando processo de Unzip das pastas')

        extension = '.zip'
        for item in os.listdir(self.dir_download): # Loop para percorrer todos os arquivos que estão na pasta "files"
            if item.endswith(extension): # Verifica se o arquivo é .zip
                #self.dir_download = os.path.abspath(self.dir_download)
                zipref = zipfile.ZipFile(self.dir_download + '\\' + item) # Criando objeto zipfile
                zipref.extractall(self.dir_download) # Extraindo arquivo para diretório
                zipref.close() # Fecha o arquivo zipado
                os.remove(self.dir_download + '\\' + item) # Deleta cada um dos zips

        print('Passo 4 de 4 - Todos os arquivos .csv foram unzipados')
        driver.quit()


    def import_df(self, dfp, extract = True):
        if extract == True:
            self.scraping_df()
        elif extract == False:
            pass
        
        print('Iniciando processo de unificação dos balanços conforme período selecionado')
        filename = []

        ini = self.ini
        fim = self.fim

        ano_ini = ini[-4:]
        ano_fim = fim[-4:]

        if ano_ini == ano_fim:
            periodo = [ano_ini, None]
        else:
            periodo = [ano_ini, ano_fim]
            lista_anos = list(range(int(periodo[0]), int(periodo[1])+1))


        for d in dfp:
            for p in lista_anos:
                filename += [x for x in os.listdir(self.dir_download) if (d in x) and (str(p) in x)]

        for i in filename:
            df = pd.read_csv(f"{self.dir_download}\\{i}", sep = ';', encoding = 'cp1252')
            try:
                df_concat = pd.concat([df_concat,df], ignore_index = True)
            except:
                df_concat = df

        df_concat['DT_INI_EXERC'] = pd.to_datetime(df_concat['DT_INI_EXERC'], format = ('%Y-%m-%d'))
        df_concat['DT_FIM_EXERC'] = pd.to_datetime(df_concat['DT_FIM_EXERC'], format = ('%Y-%m-%d'))
        df_concat['DT_REFER'] = pd.to_datetime(df_concat['DT_REFER'], format = ('%Y-%m-%d'))

        ini_log =  f'{ini[-4:]}-{ini[3:5]}-{ini[0:2]}'
        fim_log =  f'{fim[-4:]}-{fim[3:5]}-{fim[0:2]}'

        df_final = df_concat.loc[(df_concat['DT_FIM_EXERC'] >= ini_log) & (df_concat['DT_FIM_EXERC'] <= fim_log)]

        print('Formatando dados para melhor visualização')
        
        df = df_final.pivot_table(index=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER'], columns='CD_CONTA', values='VL_CONTA')
        filtro_indicadores = ['3.01', '3.02', '3.03', '3.04', '3.05', '3.06', '3.07', '3.11', '3.99']
        df_formatado = df.filter(items=filtro_indicadores, axis=1)

        indicadores_dict = {'3.01': 'Receita',
                            '3.02': 'Despes',
                            '3.03': 'Resultado Bruto',
                            '3.04': 'Despesas/Receitas Operacionais',
                            '3.05': 'EBITDA',
                            '3.06': 'Resultado Financeiro',
                            '3.07': 'EBIT',
                            '3.11':	'Lucro/Prejuízo do Período',
                            '3.99': 'Lucro por Ação'}

        df_formatado = df_formatado.rename(columns=indicadores_dict)

        print('Dataframe finalizado')
        return df_final, df_formatado
    
        

In [115]:
dre, dre_formadato = CVM_Extractor('01/01/2021','01/01/2024').import_df(['DRE'], True)

Diretório Existe
Passo 1 de 4 - Extração dos arquivos zipados da CVM
Passo 2 de 4 - Download dos arquivos concluído
Passo 3 de 4 - Iniciando processo de Unzip das pastas
Passo 4 de 4 - Todos os arquivos .csv foram unzipados
Iniciando processo de unificação dos balanços conforme período selecionado
Formatando dados para melhor visualização
Dataframe finalizado


In [116]:
dre

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.01,Receita de Venda de Bens e/ou Serviços,8.208426e+06,S
3,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-2.938607e+06,S
5,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.02.01,Energia comprada para revenda,-4.993160e+05,N
7,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.02.02,Encargos sobre uso da rede elétrica,-4.556680e+05,N
9,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.02.03,Combustível para produção de energia elétrica,-5.373370e+05,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189047,97.837.181/0001-47,2023-09-30,1,DEXCO S.A.,21091,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-07-01,2023-09-30,3.99.02,Lucro Diluído por Ação,0.000000e+00,N
1189048,97.837.181/0001-47,2023-09-30,1,DEXCO S.A.,21091,DF Individual - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2022-01-01,2022-09-30,3.99.02.01,ON,7.384000e-01,N
1189049,97.837.181/0001-47,2023-09-30,1,DEXCO S.A.,21091,DF Individual - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2022-07-01,2022-09-30,3.99.02.01,ON,2.091000e-01,N
1189050,97.837.181/0001-47,2023-09-30,1,DEXCO S.A.,21091,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-01-01,2023-09-30,3.99.02.01,ON,7.435000e-01,N


In [117]:
dre_formadato

CD_CONTA                                                   Receita  \
CNPJ_CIA           DENOM_CIA       CD_CVM DT_REFER                   
00.000.000/0001-91 BCO BRASIL S.A. 1023   2021-03-31  3.118638e+07   
                                          2021-06-30  3.172875e+07   
                                          2021-09-30  6.380887e+07   
                                          2022-03-31  3.486441e+07   
                                          2022-06-30  5.890404e+07   
...                                                            ...   
97.837.181/0001-47 DEXCO S.A.      21091  2022-06-30  2.648553e+06   
                                          2022-09-30  3.618941e+06   
                                          2023-03-31  1.664315e+06   
                                          2023-06-30  2.606599e+06   
                                          2023-09-30  3.387353e+06   

CD_CONTA                                                    Despes  \
CNPJ_CIA           DENOM_CIA       CD_CVM DT_REFER                   
00.000.000/0001-91 BCO BRASIL S.A. 1023   2021-03-31 -2.226020e+07   
                                          2021-06-30 -1.384030e+07   
                                          2021-09-30 -4.447955e+07   
                                          2022-03-31 -2.249503e+07   
                                          2022-06-30 -3.783798e+07   
...                                                            ...   
97.837.181/0001-47 DEXCO S.A.      21091  2022-06-30 -1.777975e+06   
                                          2022-09-30 -2.441286e+06   
                                          2023-03-31 -1.131760e+06   
                                          2023-06-30 -1.811617e+06   
                                          2023-09-30 -2.394193e+06   

CD_CONTA                                              Resultado Bruto  \
CNPJ_CIA           DENOM_CIA       CD_CVM DT_REFER                      
00.000.000/0001-91 BCO BRASIL S.A. 1023   2021-03-31     8.926178e+06   
                                          2021-06-30     1.788845e+07   
                                          2021-09-30     1.932933e+07   
                                          2022-03-31     1.236938e+07   
                                          2022-06-30     2.106606e+07   
...                                                               ...   
97.837.181/0001-47 DEXCO S.A.      21091  2022-06-30     8.705784e+05   
                                          2022-09-30     1.177655e+06   
                                          2023-03-31     5.325552e+05   
                                          2023-06-30     7.949819e+05   
                                          2023-09-30     9.931598e+05   

CD_CONTA                                              Despesas/Receitas Operacionais  \
CNPJ_CIA           DENOM_CIA       CD_CVM DT_REFER                                     
00.000.000/0001-91 BCO BRASIL S.A. 1023   2021-03-31                   -4.917734e+06   
                                          2021-06-30                   -9.324739e+06   
                                          2021-09-30                   -8.163990e+06   
                                          2022-03-31                   -7.216248e+06   
                                          2022-06-30                   -1.034619e+07   
...                                                                              ...   
97.837.181/0001-47 DEXCO S.A.      21091  2022-06-30                   -1.282146e+05   
                                          2022-09-30                   -3.768995e+05   
                                          2023-03-31                   -1.899555e+05   
                                          2023-06-30                   -3.260616e+05   
                                          2023-09-30                   -3.895985e+05   

CD_CONTA                                                    EBITDA  \
CNPJ_CIA     

In [124]:
dre_formadato.loc['00.000.000/0001-91']

CD_CONTA                                Receita        Despes  \
DENOM_CIA       CD_CVM DT_REFER                                 
BCO BRASIL S.A. 1023   2021-03-31  3.118638e+07 -2.226020e+07   
                       2021-06-30  3.172875e+07 -1.384030e+07   
                       2021-09-30  6.380887e+07 -4.447955e+07   
                       2022-03-31  3.486441e+07 -2.249503e+07   
                       2022-06-30  5.890404e+07 -3.783798e+07   
                       2022-09-30  9.002921e+07 -6.216640e+07   
                       2023-03-31  5.231057e+07 -3.692390e+07   
                       2023-06-30  8.872957e+07 -6.444967e+07   
                       2023-09-30  1.264390e+08 -9.304831e+07   

CD_CONTA                           Resultado Bruto  \
DENOM_CIA       CD_CVM DT_REFER                      
BCO BRASIL S.A. 1023   2021-03-31     8.926178e+06   
                       2021-06-30     1.788845e+07   
                       2021-09-30     1.932933e+07   
                       2022-03-31     1.236938e+07   
                       2022-06-30     2.106606e+07   
                       2022-09-30     2.786281e+07   
                       2023-03-31     1.538667e+07   
                       2023-06-30     2.427990e+07   
                       2023-09-30     3.339069e+07   

CD_CONTA                           Despesas/Receitas Operacionais  \
DENOM_CIA       CD_CVM DT_REFER                                     
BCO BRASIL S.A. 1023   2021-03-31                   -4.917734e+06   
                       2021-06-30                   -9.324739e+06   
                       2021-09-30                   -8.163990e+06   
                       2022-03-31                   -7.216248e+06   
                       2022-06-30                   -1.034619e+07   
                       2022-09-30                   -1.303581e+07   
                       2023-03-31                   -7.314910e+06   
                       2023-06-30                   -9.342220e+06   
                       2023-09-30                   -1.307961e+07   

CD_CONTA                                 EBITDA  Resultado Financeiro  \
DENOM_CIA       CD_CVM DT_REFER                                         
BCO BRASIL S.A. 1023   2021-03-31  4.008444e+06            666660.000   
                       2021-06-30  8.563714e+06             71586.000   
                       2021-09-30  1.116534e+07           -546915.000   
                       2022-03-31  5.153130e+06            122453.500   
                       2022-06-30  1.071987e+07          -1104248.250   
                       2022-09-30  1.482699e+07          -2156958.000   
                       2023-03-31  8.071761e+06           -846743.500   
                       2023-06-30  1.493768e+07          -2342412.000   
                       2023-09-30  2.031108e+07          -3314244.375   

CD_CONTA                                   EBIT  Lucro/Prejuízo do Período  \
DENOM_CIA       CD_CVM DT_REFER                                              
BCO BRASIL S.A. 1023   2021-03-31  4.675104e+06               4.157202e+06   
                       2021-06-30  8.635300e+06               7.645832e+06   
                       2021-09-30  1.061842e+07               9.403716e+06   
                       2022-03-31  5.275584e+06               4.933498e+06   
                       2022-06-30  9.615619e+06               9.018788e+06   
                       2022-09-30  1.267004e+07               1.188027e+07   
                       2023-03-31  7.225018e+06               6.750808e+06   
                       2023-06-30  1.259527e+07               1.184815e+07   
                       2023-09-30  1.699684e+07               1.597862e+07   

CD_CONTA                           Lucro por Ação  
DENOM_CIA       CD_CVM DT_REFER                    
BCO BRASIL S.A. 1023   2021-03-31         1.46000  
                       2021-06-30         2.68000  
                       2021-09-30         3.29500  
      